In [12]:
import numpy as np
from tqdm import tqdm
import time
from numba import njit, prange, jit
from typing import NamedTuple
import zlib
from collections import deque

In [13]:
# Create a named Tuple class to store the problems
class ProblemInstance(NamedTuple):
    holding_costs: np.ndarray
    setup_costs: np.ndarray
    setup_times: np.ndarray
    demand: np.ndarray
    production_times: np.ndarray
    capacities: np.ndarray
    num_periods: int
    num_products: int

# Create a class for the tabu list
class TabuList:
    def __init__(self, max_size=1000):

        # Initialze the hashes, FIFO queue and max size of the list
        self.hashes = set()
        self.queue = deque()
        self.max_size = max_size

    def hash_solution(self, sol: np.ndarray):
        # Flatten the solution and convert to bytes
        byte_repr = sol.tobytes()
        return zlib.crc32(byte_repr)

    def is_tabu(self, sol: np.ndarray):
        h = self.hash_solution(sol)
        # If the solution exists within the tabu list, return true
        return h in self.hashes

    def add(self, sol: np.ndarray):
        h = self.hash_solution(sol)
        if len(self.queue) >= self.max_size:
            old = self.queue.popleft()
            self.hashes.remove(old)
        self.queue.append(h)
        self.hashes.add(h)

In [14]:
dem = np.array([[0, 0, 0, 39, 26, 50, 38, 0],
                [0, 0, 3, 0, 45, 42, 10, 29],
                [0, 50, 30, 0, 23, 0, 0, 20],
                [0, 37, 0, 40, 0, 36, 0, 0],
                [200, 39, 39, 10, 0, 43, 30, 0]],dtype=np.int32)
demand = []

for i in range(5):

    for x in range(5):
        demand.append([0 for i in range(8)])
    demand.append(dem[i])

demand = np.array(demand,dtype=np.int32)
problem = ProblemInstance(
    holding_costs = np.array([1,1,1,1,1,1,1,1],dtype=np.int32),
    setup_costs = np.array([[0, 152, 139, 143, 196, 113, 210, 110],
                            [152, 0, 105, 195, 196, 127, 150, 175],
                            [139, 105, 0, 108, 198, 171, 125, 125],
                            [143, 195, 108, 0, 117, 157, 120, 176],
                            [196, 196, 198, 117, 0, 162, 220, 177],
                            [113, 127, 171, 157, 162, 0, 120, 130],
                            [210, 150, 125, 120, 220, 120, 0, 200],
                            [110, 175, 125, 176, 177, 130, 200, 0]],dtype=np.int32),
setup_times = np.array([[0, 11, 18, 18, 11, 13, 15, 18],
                        [11, 0, 15, 13, 17, 14, 16, 11],
                        [18, 15, 0, 16, 12, 12, 15, 11],
                        [18, 13, 16, 0, 18, 16, 10, 10],
                        [11, 17, 12, 18, 0, 20, 20, 19],
                        [13, 14, 12, 16, 20, 0, 15, 12],
                        [15, 16, 15, 10, 20, 15, 0, 12],
                        [18, 11, 11, 10, 19, 12, 12, 0]],dtype=np.int32),
 demand = demand,

 production_times = np.array([1,1,1,1,1,1,1,1]),
 
 capacities = np.array([38 for i in range(30)],dtype=np.int32),
        num_periods = 30,
        num_products = 8)

In [15]:
@njit
def decode_full(demand, capacities, production_times, setup_times,
           solution_full, holding_costs, setup_costs, penalty_shortage=1):

    # ========== BACKLOG MATRIX ==========

    periods, products = demand.shape
    Skip_production = products
    setup_change = products + 1
    setup_change_direct = products + 2
    solution = solution_full[:, :-3]

    # Create the backlog matrix and a matrix for the cumulative demand
    backlog = np.zeros((periods, products), dtype=np.int32)
    cum_demand = np.zeros(products, dtype=np.int32)

    # For each period we add the demand to the cum_demand array which subsequently gets added to the backlog matrix
    # The backlog matrix shows (beginning with the last period) how many products still need to be produced
    for per in range(periods):
        src = periods - per - 1
        for prod in range(products):
            cum_demand[prod] += demand[src, prod]
        for prod in range(products):
            backlog[src, prod] = cum_demand[prod]

    # ========== LAST PERIOD ==========
            
    # Create a matrix for the decoded solution
    decoded = np.zeros((periods + 1, 3), dtype=np.int32)
    

    # Get the index of the last period
    period = periods - 1

    # Get the valid products (those with backlog > 0)
    valid_products = np.full(products, -1.0)


    # If there is demand in the previous period, but not in this one, we can change the setup
    if np.all(backlog[period] == 0):
        for p in range(products):
            valid_products[p] = solution[period, p]
    else:
        for p in range(products):
            if backlog[period, p] > 0:
                valid_products[p] = solution[period, p]

    # Select the product with the highest priority
    second_product = np.argmax(valid_products)
    decoded[period+1, 2] = second_product


    capacity = capacities[period]

    # If the binary indicator for a setup change is 1, directly plan the setup change so that we do not violate capacity restrictions
    if solution_full[period, setup_change] == 1:

        # Valid products this time also include those from the next period
        back = backlog[period] + backlog[period - 1]
        valid_products_2 = np.full(products, -1.0)
        for p in range(products):
            if back[p] > 0:
                valid_products_2[p] = solution[period, p]

        # We set the value of the first product to -1, so that it cannot be selected
        valid_products_2[second_product] = -1
        first_product = np.argmax(valid_products_2)

        # Set the first product and subtract the setup times from the capcity
        decoded[period, 2] = first_product
        capacity -= setup_times[first_product, second_product]

        # Produce the second product either until the demand or capacity is satisfied
        max_quantity = capacity // production_times[second_product]
        quantity = min(max_quantity, backlog[period, second_product])
        decoded[period + 1, 0] = quantity
        for j in range(periods):
            backlog[j, second_product] -= quantity

        capacity -= quantity * production_times[second_product]

        # Produce the first product if there is spare capacity
        if capacity > 0 and solution_full[period, Skip_production] == 0:
            max_quantity = capacity // production_times[first_product]
            quantity = min(max_quantity, backlog[period, first_product])
            decoded[period, 1] = quantity
            for j in range(periods):
                backlog[j, first_product] -= quantity

    # If we do not change the setup, just produce the second product 
    else:
        max_quantity = capacity // production_times[second_product]
        quantity = min(max_quantity, backlog[period, second_product])
        decoded[period + 1, 0] = quantity
        for j in range(periods):
            backlog[j, second_product] -= quantity

        decoded[period, 2] = second_product



    # ========== OTHER PERIODS ==========
            
    for i in range(len(solution) - 1):

        # Start with the second to last period and work to the beginning
        period = periods - 2 - i

        # Get the valid products (those with backlog > 0)
        valid_products = np.full(products, -1.0)
        # If there is no demand in this or the previous period, we do not need to produce or change the setup
        if np.all(backlog[period] + backlog[period-1] == 0):
            decoded[period,2] = decoded[period+1,2]
            continue


        # If there is demand in the previous period, but not in this one, we can change the setup
        elif np.all(backlog[period] == 0):
            for p in range(products):
                valid_products[p] = solution[period, p]
        else:
            for p in range(products):
                if backlog[period, p] > 0:
                    valid_products[p] = solution[period, p]

        # Select the product with the highest priority
        second_product = np.argmax(valid_products)

        # If the product with the highest priority is the same as in the next period, we do not need to change the setup
        if second_product == decoded[period + 1, 2]:
            capacity = capacities[period]

            # If the binary indicator for a setup change is 1, directly plan the setup change so that we do not violate capacity restrictions
            if solution_full[period, setup_change] == 1:

                # Valid products this time also include those from the next period
                back = backlog[period] + backlog[period - 1]
                valid_products_2 = np.full(products, -1.0)
                for p in range(products):
                    if back[p] > 0:
                        valid_products_2[p] = solution[period, p]

                # We set the value of the first product to -1, so that it cannot be selected
                valid_products_2[second_product] = -1
                first_product = np.argmax(valid_products_2)

                # Set the first product and subtract the setup times from the capcity
                decoded[period, 2] = first_product
                capacity -= setup_times[first_product, second_product]

                # Produce the second product either until the demand or capacity is satisfied
                max_quantity = capacity // production_times[second_product]
                quantity = min(max_quantity, backlog[period, second_product])
                decoded[period + 1, 0] = quantity
                for j in range(periods):
                    backlog[j, second_product] -= quantity

                capacity -= quantity * production_times[second_product]

                # Produce the first product if there is spare capacity
                if capacity > 0 and solution_full[period, Skip_production] == 0:
                    max_quantity = capacity // production_times[first_product]
                    quantity = min(max_quantity, backlog[period, first_product])
                    decoded[period, 1] = quantity
                    for j in range(periods):
                        backlog[j, first_product] -= quantity

            # If we do not change the setup, just produce the second product 
            else:
                max_quantity = capacity // production_times[second_product]
                quantity = min(max_quantity, backlog[period, second_product])
                decoded[period + 1, 0] = quantity
                for j in range(periods):
                    backlog[j, second_product] -= quantity

                decoded[period, 2] = second_product

        elif solution_full[period, setup_change_direct] == 1:

            if backlog[period, decoded[period+1, 2]] > 0:
                second_product = decoded[period+1, 2]
                decoded[period, 2] = second_product
                spare_capacity = capacities[period] - setup_times[second_product, decoded[period + 1, 2]]
                max_quantity = spare_capacity // production_times[second_product]
                quantity = min(max_quantity, backlog[period, second_product])
                decoded[period, 1] = quantity
                for j in range(periods):
                    backlog[j, second_product] -= quantity

            elif solution_full[period, Skip_production] == 0:
                decoded[period, 2] = second_product
                spare_capacity = capacities[period] - setup_times[second_product, decoded[period + 1, 2]]
                max_quantity = spare_capacity // production_times[second_product]
                quantity = min(max_quantity, backlog[period, second_product])
                decoded[period, 1] = quantity
                for j in range(periods):
                    backlog[j, second_product] -= quantity

            else:
                decoded[period, 2] = decoded[period + 1, 2]


        # If the product with the highest priority differs from the one in the last period, change the setup directly
        elif solution_full[period, Skip_production] == 0:
            decoded[period, 2] = second_product
            spare_capacity = capacities[period] - setup_times[second_product, decoded[period + 1, 2]]
            max_quantity = spare_capacity // production_times[second_product]
            quantity = min(max_quantity, backlog[period, second_product])
            decoded[period, 1] = quantity
            for j in range(periods):
                backlog[j, second_product] -= quantity

        # If we skip production, just transfer the setup from the last period
        else:
            decoded[period, 2] = decoded[period + 1, 2]

    # If the backlog matrix is > 0, the solution is infeasible and must be punished
    shortage = np.sum(backlog[0]) * penalty_shortage

    # ========== CALCULATE FITNESS ==========
    lots1 = decoded[1:, 0]
    lots2 = decoded[:-1, 1]
    seq = decoded[:, 2]

    # Create a matrix with the production quantites (Periods x Products)
    production = np.zeros((periods, products), dtype=np.int32)
    for t in range(periods):
        production[t, seq[t + 1]] += lots1[t]
        production[t, seq[t]] += lots2[t]

    # Add the production quantities up to get the inventory matrix (Periods x Products)
    inventory = np.zeros((periods, products), dtype=np.int32)
    for m in range(products):
        cum = 0
        for t in range(periods):
            cum += production[t, m] - demand[t, m]
            inventory[t, m] = cum

    # Calaculate holding and shortage costs
    hold_cost = 0
    for t in range(periods):
        for m in range(products):
            inv = inventory[t, m]
            hold_cost += max(inv,0) * holding_costs[m]

    #print(inventory)

    setup_cost = 0
    for t in range(periods):
        setup_cost += setup_costs[seq[t], seq[t + 1]]

    total_cost = hold_cost + shortage + setup_cost

    return decoded, total_cost

def decode_verbose(demand, capacities, production_times, setup_times,
           solution_full, holding_costs, setup_costs, penalty_shortage=1):

    # ========== BACKLOG MATRIX ==========

    periods, products = demand.shape
    Skip_production = products
    setup_change = products + 1
    setup_change_direct = products + 2
    solution = solution_full[:, :-3]

    # Create the backlog matrix and a matrix for the cumulative demand
    backlog = np.zeros((periods, products), dtype=np.int32)
    cum_demand = np.zeros(products, dtype=np.int32)

    # For each period we add the demand to the cum_demand array which subsequently gets added to the backlog matrix
    # The backlog matrix shows (beginning with the last period) how many products still need to be produced
    for per in range(periods):
        src = periods - per - 1
        for prod in range(products):
            cum_demand[prod] += demand[src, prod]
        for prod in range(products):
            backlog[src, prod] = cum_demand[prod]

    # ========== LAST PERIOD ==========
            
    # ========== LAST PERIOD ==========
            
    # Create a matrix for the decoded solution
    decoded = np.zeros((periods + 1, 3), dtype=np.int32)
    

    # Get the index of the last period
    period = periods - 1

    # Get the valid products (those with backlog > 0)
    valid_products = np.full(products, -1.0)


    # If there is demand in the previous period, but not in this one, we can change the setup
    if np.all(backlog[period] == 0):
        for p in range(products):
            valid_products[p] = solution[period, p]
    else:
        for p in range(products):
            if backlog[period, p] > 0:
                valid_products[p] = solution[period, p]

    # Select the product with the highest priority
    second_product = np.argmax(valid_products)
    decoded[period+1, 2] = second_product


    capacity = capacities[period]

    # If the binary indicator for a setup change is 1, directly plan the setup change so that we do not violate capacity restrictions
    if solution_full[period, setup_change] == 1:

        # Valid products this time also include those from the next period
        back = backlog[period] + backlog[period - 1]
        valid_products_2 = np.full(products, -1.0)
        for p in range(products):
            if back[p] > 0:
                valid_products_2[p] = solution[period, p]

        # We set the value of the first product to -1, so that it cannot be selected
        valid_products_2[second_product] = -1
        first_product = np.argmax(valid_products_2)

        # Set the first product and subtract the setup times from the capcity
        decoded[period, 2] = first_product
        capacity -= setup_times[first_product, second_product]

        # Produce the second product either until the demand or capacity is satisfied
        max_quantity = capacity // production_times[second_product]
        quantity = min(max_quantity, backlog[period, second_product])
        decoded[period + 1, 0] = quantity
        for j in range(periods):
            backlog[j, second_product] -= quantity

        capacity -= quantity * production_times[second_product]

        # Produce the first product if there is spare capacity
        if capacity > 0 and solution_full[period, Skip_production] == 0:
            max_quantity = capacity // production_times[first_product]
            quantity = min(max_quantity, backlog[period, first_product])
            decoded[period, 1] = quantity
            for j in range(periods):
                backlog[j, first_product] -= quantity

    # If we do not change the setup, just produce the second product 
    else:
        max_quantity = capacity // production_times[second_product]
        quantity = min(max_quantity, backlog[period, second_product])
        decoded[period + 1, 0] = quantity
        for j in range(periods):
            backlog[j, second_product] -= quantity

        decoded[period, 2] = second_product

    # ========== OTHER PERIODS ==========
            
    for i in range(len(solution) - 1):

        # Start with the second to last period and work to the beginning
        period = periods - 2 - i

        # Get the valid products (those with backlog > 0)
        valid_products = np.full(products, -1.0)
        # If there is no demand in this or the previous period, we do not need to produce or change the setup
        if np.all(backlog[period] + backlog[period-1] == 0):
            decoded[period,2] = decoded[period+1,2]
            continue


        # If there is demand in the previous period, but not in this one, we can change the setup
        elif np.all(backlog[period] == 0):
            for p in range(products):
                valid_products[p] = solution[period, p]
        else:
            for p in range(products):
                if backlog[period, p] > 0:
                    valid_products[p] = solution[period, p]

        # Select the product with the highest priority
        second_product = np.argmax(valid_products)

        # If the product with the highest priority is the same as in the next period, we do not need to change the setup
        if second_product == decoded[period + 1, 2]:
            capacity = capacities[period]

            # If the binary indicator for a setup change is 1, directly plan the setup change so that we do not violate capacity restrictions
            if solution_full[period, setup_change] == 1:

                # Valid products this time also include those from the next period
                back = backlog[period] + backlog[period - 1]
                valid_products_2 = np.full(products, -1.0)
                for p in range(products):
                    if back[p] > 0:
                        valid_products_2[p] = solution[period, p]

                # We set the value of the first product to -1, so that it cannot be selected
                valid_products_2[second_product] = -1
                first_product = np.argmax(valid_products_2)

                # Set the first product and subtract the setup times from the capcity
                decoded[period, 2] = first_product
                capacity -= setup_times[first_product, second_product]

                # Produce the second product either until the demand or capacity is satisfied
                max_quantity = capacity // production_times[second_product]
                quantity = min(max_quantity, backlog[period, second_product])
                decoded[period + 1, 0] = quantity
                for j in range(periods):
                    backlog[j, second_product] -= quantity

                capacity -= quantity * production_times[second_product]

                # Produce the first product if there is spare capacity
                if capacity > 0 and solution_full[period, Skip_production] == 0:
                    max_quantity = capacity // production_times[first_product]
                    quantity = min(max_quantity, backlog[period, first_product])
                    decoded[period, 1] = quantity
                    for j in range(periods):
                        backlog[j, first_product] -= quantity

            # If we do not change the setup, just produce the second product 
            else:
                max_quantity = capacity // production_times[second_product]
                quantity = min(max_quantity, backlog[period, second_product])
                decoded[period + 1, 0] = quantity
                for j in range(periods):
                    backlog[j, second_product] -= quantity

                decoded[period, 2] = second_product

        elif solution_full[period, setup_change_direct] == 1:

            if backlog[period, decoded[period+1, 2]] > 0:
                second_product = decoded[period+1, 2]
                decoded[period, 2] = second_product
                spare_capacity = capacities[period] - setup_times[second_product, decoded[period + 1, 2]]
                max_quantity = spare_capacity // production_times[second_product]
                quantity = min(max_quantity, backlog[period, second_product])
                decoded[period, 1] = quantity
                for j in range(periods):
                    backlog[j, second_product] -= quantity

            elif solution_full[period, Skip_production] == 0:
                decoded[period, 2] = second_product
                spare_capacity = capacities[period] - setup_times[second_product, decoded[period + 1, 2]]
                max_quantity = spare_capacity // production_times[second_product]
                quantity = min(max_quantity, backlog[period, second_product])
                decoded[period, 1] = quantity
                for j in range(periods):
                    backlog[j, second_product] -= quantity

            else:
                decoded[period, 2] = decoded[period + 1, 2]


        # If the product with the highest priority differs from the one in the last period, change the setup directly
        elif solution_full[period, Skip_production] == 0:
            decoded[period, 2] = second_product
            spare_capacity = capacities[period] - setup_times[second_product, decoded[period + 1, 2]]
            max_quantity = spare_capacity // production_times[second_product]
            quantity = min(max_quantity, backlog[period, second_product])
            decoded[period, 1] = quantity
            for j in range(periods):
                backlog[j, second_product] -= quantity

        # If we skip production, just transfer the setup from the last period
        else:
            decoded[period, 2] = decoded[period + 1, 2]

    periods, products = demand.shape

    lots1 = decoded[1:, 0].astype(np.int32)
    lots2 = decoded[:-1, 1].astype(np.int32)
    seq = decoded[:, 2].astype(np.int32)

    # --- Production matrix ---
    production = np.zeros((periods, products), dtype=np.int32)
    for t in range(periods):
        production[t, seq[t + 1]] += lots1[t]
        production[t, seq[t]] += lots2[t]

    print("\n=== Production Matrix ===")
    print(production)

    # --- Inventory matrix ---
    inventory = np.zeros((periods, products), dtype=np.float64)
    for m in range(products):
        cum = 0.0
        for t in range(periods):
            cum += production[t, m] - demand[t, m]
            inventory[t, m] = cum

    print("\n=== Inventory Matrix ===")
    print(inventory)

    # --- Holding and shortage costs ---
    hold_costs_per_product = np.sum(inventory * holding_costs, axis=0)
    hold_cost_total = np.sum(hold_costs_per_product)

    shortage_violations = inventory < 0
    shortage_units = -np.sum(inventory[shortage_violations])
    shortage_penalty = shortage_units * penalty_shortage

    print("\n--- Holding Costs per Product ---")
    for i in range(products):
        print(f"Product {i}: {hold_costs_per_product[i]:.2f}")

    print("\nTotal Holding Cost:", hold_cost_total)
    print("Total Shortage Penalty:", shortage_penalty)
    print("Total Shortage Violations (units):", shortage_units)

    # --- Setup and production time ---
    setup_time = np.zeros(periods, dtype=np.int32)
    for t in range(periods):
        setup_time[t] = setup_times[seq[t], seq[t + 1]]

    prod_time = np.sum(production * production_times, axis=1)
    total_time = prod_time + setup_time
    overtime_violations = np.maximum(total_time - capacities, 0)

    print("\nSetup Times per Period:", setup_time)
    print("Production Times per Period:", prod_time)
    print("Total Time per Period:", total_time)
    print("Overtime Violations (units):", np.sum(overtime_violations))

    # --- Setup cost ---
    setup_c = 0.0
    for t in range(periods):
        setup_c += setup_costs[seq[t], seq[t + 1]]

    print("\nTotal Setup Cost:", setup_c)

    total_cost = hold_cost_total + shortage_penalty + setup_c
    print("\n=== Final Total Cost ===")
    if shortage_penalty == 0:
        print(f'\nFeasible Solution found with Total Cost: {total_cost}')
    else:
        print(f'\nNo feasible Solution found')

    return decoded

@njit
def decode_prod_plan(demand, capacities, production_times, setup_times,
           solution_full, holding_costs, setup_costs, penalty_shortage=1):

    # ========== BACKLOG MATRIX ==========

    periods, products = demand.shape
    Skip_production = products
    setup_change = products + 1
    setup_change_direct = products + 2
    solution = solution_full[:, :-3]

    # Create the backlog matrix and a matrix for the cumulative demand
    backlog = np.zeros((periods, products), dtype=np.int32)
    cum_demand = np.zeros(products, dtype=np.int32)

    # For each period we add the demand to the cum_demand array which subsequently gets added to the backlog matrix
    # The backlog matrix shows (beginning with the last period) how many products still need to be produced
    for per in range(periods):
        src = periods - per - 1
        for prod in range(products):
            cum_demand[prod] += demand[src, prod]
        for prod in range(products):
            backlog[src, prod] = cum_demand[prod]

    # ========== LAST PERIOD ==========
            
    # Create a matrix for the decoded solution
    decoded = np.zeros((periods + 1, 3), dtype=np.int32)
    

    # Get the index of the last period
    period = periods - 1

    # Get the valid products (those with backlog > 0)
    valid_products = np.full(products, -1.0)


    # If there is demand in the previous period, but not in this one, we can change the setup
    if np.all(backlog[period] == 0):
        for p in range(products):
            valid_products[p] = solution[period, p]
    else:
        for p in range(products):
            if backlog[period, p] > 0:
                valid_products[p] = solution[period, p]

    # Select the product with the highest priority
    second_product = np.argmax(valid_products)
    decoded[period+1, 2] = second_product


    capacity = capacities[period]

    # If the binary indicator for a setup change is 1, directly plan the setup change so that we do not violate capacity restrictions
    if solution_full[period, setup_change] == 1:

        # Valid products this time also include those from the next period
        back = backlog[period] + backlog[period - 1]
        valid_products_2 = np.full(products, -1.0)
        for p in range(products):
            if back[p] > 0:
                valid_products_2[p] = solution[period, p]

        # We set the value of the first product to -1, so that it cannot be selected
        valid_products_2[second_product] = -1
        first_product = np.argmax(valid_products_2)

        # Set the first product and subtract the setup times from the capcity
        decoded[period, 2] = first_product
        capacity -= setup_times[first_product, second_product]

        # Produce the second product either until the demand or capacity is satisfied
        max_quantity = capacity // production_times[second_product]
        quantity = min(max_quantity, backlog[period, second_product])
        decoded[period + 1, 0] = quantity
        for j in range(periods):
            backlog[j, second_product] -= quantity

        capacity -= quantity * production_times[second_product]

        # Produce the first product if there is spare capacity
        if capacity > 0 and solution_full[period, Skip_production] == 0:
            max_quantity = capacity // production_times[first_product]
            quantity = min(max_quantity, backlog[period, first_product])
            decoded[period, 1] = quantity
            for j in range(periods):
                backlog[j, first_product] -= quantity

    # If we do not change the setup, just produce the second product 
    else:
        max_quantity = capacity // production_times[second_product]
        quantity = min(max_quantity, backlog[period, second_product])
        decoded[period + 1, 0] = quantity
        for j in range(periods):
            backlog[j, second_product] -= quantity

        decoded[period, 2] = second_product



    # ========== OTHER PERIODS ==========
            
    for i in range(len(solution) - 1):

        # Start with the second to last period and work to the beginning
        period = periods - 2 - i

        # Get the valid products (those with backlog > 0)
        valid_products = np.full(products, -1.0)
        # If there is no demand in this or the previous period, we do not need to produce or change the setup
        if np.all(backlog[period] + backlog[period-1] == 0):
            decoded[period,2] = decoded[period+1,2]
            continue


        # If there is demand in the previous period, but not in this one, we can change the setup
        elif np.all(backlog[period] == 0):
            for p in range(products):
                valid_products[p] = solution[period, p]
        else:
            for p in range(products):
                if backlog[period, p] > 0:
                    valid_products[p] = solution[period, p]

        # Select the product with the highest priority
        second_product = np.argmax(valid_products)

        # If the product with the highest priority is the same as in the next period, we do not need to change the setup
        if second_product == decoded[period + 1, 2]:
            capacity = capacities[period]

            # If the binary indicator for a setup change is 1, directly plan the setup change so that we do not violate capacity restrictions
            if solution_full[period, setup_change] == 1:

                # Valid products this time also include those from the next period
                back = backlog[period] + backlog[period - 1]
                valid_products_2 = np.full(products, -1.0)
                for p in range(products):
                    if back[p] > 0:
                        valid_products_2[p] = solution[period, p]

                # We set the value of the first product to -1, so that it cannot be selected
                valid_products_2[second_product] = -1
                first_product = np.argmax(valid_products_2)

                # Set the first product and subtract the setup times from the capcity
                decoded[period, 2] = first_product
                capacity -= setup_times[first_product, second_product]

                # Produce the second product either until the demand or capacity is satisfied
                max_quantity = capacity // production_times[second_product]
                quantity = min(max_quantity, backlog[period, second_product])
                decoded[period + 1, 0] = quantity
                for j in range(periods):
                    backlog[j, second_product] -= quantity

                capacity -= quantity * production_times[second_product]

                # Produce the first product if there is spare capacity
                if capacity > 0 and solution_full[period, Skip_production] == 0:
                    max_quantity = capacity // production_times[first_product]
                    quantity = min(max_quantity, backlog[period, first_product])
                    decoded[period, 1] = quantity
                    for j in range(periods):
                        backlog[j, first_product] -= quantity

            # If we do not change the setup, just produce the second product 
            else:
                max_quantity = capacity // production_times[second_product]
                quantity = min(max_quantity, backlog[period, second_product])
                decoded[period + 1, 0] = quantity
                for j in range(periods):
                    backlog[j, second_product] -= quantity

                decoded[period, 2] = second_product

        elif solution_full[period, setup_change_direct] == 1:

            if backlog[period, decoded[period+1, 2]] > 0:
                second_product = decoded[period+1, 2]
                decoded[period, 2] = second_product
                spare_capacity = capacities[period] - setup_times[second_product, decoded[period + 1, 2]]
                max_quantity = spare_capacity // production_times[second_product]
                quantity = min(max_quantity, backlog[period, second_product])
                decoded[period, 1] = quantity
                for j in range(periods):
                    backlog[j, second_product] -= quantity

            elif solution_full[period, Skip_production] == 0:
                decoded[period, 2] = second_product
                spare_capacity = capacities[period] - setup_times[second_product, decoded[period + 1, 2]]
                max_quantity = spare_capacity // production_times[second_product]
                quantity = min(max_quantity, backlog[period, second_product])
                decoded[period, 1] = quantity
                for j in range(periods):
                    backlog[j, second_product] -= quantity

            else:
                decoded[period, 2] = decoded[period + 1, 2]


        # If the product with the highest priority differs from the one in the last period, change the setup directly
        elif solution_full[period, Skip_production] == 0:
            decoded[period, 2] = second_product
            spare_capacity = capacities[period] - setup_times[second_product, decoded[period + 1, 2]]
            max_quantity = spare_capacity // production_times[second_product]
            quantity = min(max_quantity, backlog[period, second_product])
            decoded[period, 1] = quantity
            for j in range(periods):
                backlog[j, second_product] -= quantity

        # If we skip production, just transfer the setup from the last period
        else:
            decoded[period, 2] = decoded[period + 1, 2]

    return decoded

    

In [16]:
sol = np.random.rand(30,8)
sol2 = np.random.randint(2,size=(30,3))
solution = np.hstack([sol,sol2])

In [17]:
new_sol = decode_prod_plan(problem.demand,problem.capacities,problem.production_times,problem.setup_times,solution,problem.holding_costs,problem.setup_costs,1000)

In [18]:
def run_tabu_iteration(problem: ProblemInstance,
                       solution: np.ndarray,
                       neighbourhood_size: int,
                       penalty: float,
                       tabu_list: TabuList):

    scratch = np.empty_like(solution)
    num_periods = problem.num_periods
    num_products = problem.num_products

    neighbourhood = np.zeros((neighbourhood_size, num_periods, num_products + 3), dtype=np.float64)
    neighbourhood_fit = np.zeros(neighbourhood_size, dtype=np.float64)
    valid_count = 0

    for period in range(num_periods):
        for i in range(num_products):
            np.copyto(scratch, solution)
            active_row = scratch[period, :-3]

            max_prio = active_row.argmax()
            if max_prio != i:
                active_row[i], active_row[max_prio] = active_row[max_prio], active_row[i]

                scratch_decode, fitness = decode_full(problem.demand, problem.capacities,
                                                      problem.production_times, problem.setup_times,
                                                      scratch, problem.holding_costs, problem.setup_costs, penalty)

                if not tabu_list.is_tabu(scratch_decode):
                    neighbourhood[valid_count] = scratch
                    neighbourhood_fit[valid_count] = fitness
                    valid_count += 1

        for i in range(3):
            np.copyto(scratch, solution)
            scratch[period, -3 + i] = 1.0 - scratch[period, -3 + i]

            scratch_decode, fitness = decode_full(problem.demand, problem.capacities,
                                                  problem.production_times, problem.setup_times,
                                                  scratch, problem.holding_costs, problem.setup_costs, penalty)

            if tabu_list.is_tabu(scratch_decode) == False:
                neighbourhood[valid_count] = scratch
                neighbourhood_fit[valid_count] = fitness
                valid_count += 1

    if valid_count == 0:
        return solution.copy(), -1, False
    
    idx = np.argmin(neighbourhood_fit[:valid_count])
    best_neighbour = neighbourhood[idx]
    best_neighbour_fit = neighbourhood_fit[idx]

    return best_neighbour, best_neighbour_fit, True

In [19]:
def run_tabu_search(problem:ProblemInstance,
                    start_solution:np.ndarray,
                    start_solution_decoded:np.ndarray,
                    start_obj_value:int,
                    len_tabu_list:int,
                    num_iterations:int,
                    penalty:int):

    # Create a variable that always stores the current best solution/objective value
    best_solution = start_solution
    best_objective_value = start_obj_value

    # Store the current solution
    current_solution = start_solution

    # Create the Tabu List
    tabu_list = TabuList(len_tabu_list)
    tabu_list.add(start_solution_decoded)

    # Get the number of periods and products
    periods = problem.num_periods
    products = problem.num_products

    # Create a progress bar
    progress = tqdm(total=num_iterations, desc="Optimizing", dynamic_ncols=True)

    # Calculate the size of the neighbourhood
    neighbourhood_size = periods * ((products-1) + 3)

    for iter in range(num_iterations):

        # Run one iteration of the algorithm
        current_solution, best_neighbour_fit, valid = run_tabu_iteration(problem,current_solution,neighbourhood_size,penalty,tabu_list)

        if not valid:
            decode_verbose(problem.demand,problem.capacities,problem.production_times,problem.setup_times,
                    best_solution,problem.holding_costs,problem.setup_costs,penalty)
            break

        # Add the best neighbour to the tabu list
        best_neighbour_decode = decode_prod_plan(problem.demand,
                                                 problem.capacities,
                                                 problem.production_times,
                                                 problem.setup_times,
                                                 current_solution,
                                                 problem.holding_costs,
                                                 problem.setup_costs,
                                                 penalty)       
        tabu_list.add(best_neighbour_decode)

        # Update the global best, if the best neighbour is better
        if best_neighbour_fit < best_objective_value:
            best_objective_value = best_neighbour_fit
            best_solution = current_solution


        # Update the progress bar
        progress.update(1)
        progress.set_description(f"Iteration: {iter+1}, Best: {best_objective_value:.0f}, Current: {best_neighbour_fit:.0f}")



    
    decode_verbose(problem.demand,problem.capacities,problem.production_times,problem.setup_times,
                    best_solution,problem.holding_costs,problem.setup_costs,penalty)   
        

        





In [20]:
sol = np.random.rand(30,8)
sol2 = np.random.randint(2,size=(30,3))
solution = np.hstack([sol,sol2])
solution

array([[0.34992656, 0.9075887 , 0.74060473, 0.30435865, 0.79512608,
        0.16725448, 0.58900633, 0.72074046, 1.        , 0.        ,
        1.        ],
       [0.37715981, 0.2565619 , 0.04930712, 0.50715421, 0.41715919,
        0.59714119, 0.70918324, 0.60709033, 0.        , 0.        ,
        0.        ],
       [0.56105073, 0.40179439, 0.09154115, 0.15756542, 0.59995207,
        0.66310215, 0.09896202, 0.07213092, 1.        , 0.        ,
        1.        ],
       [0.2594809 , 0.4439068 , 0.29477304, 0.33945498, 0.50313073,
        0.60750513, 0.14398166, 0.44809727, 0.        , 0.        ,
        1.        ],
       [0.45241113, 0.3061329 , 0.76234782, 0.46252403, 0.9830757 ,
        0.95665452, 0.37361867, 0.38610969, 0.        , 0.        ,
        0.        ],
       [0.63966521, 0.69329041, 0.35712692, 0.18495261, 0.92444795,
        0.58200743, 0.88213082, 0.58753265, 0.        , 1.        ,
        0.        ],
       [0.42887493, 0.49665763, 0.42695356, 0.3554246 , 0.

In [21]:
sol_decode, sol_fit = decode_full(problem.demand,problem.capacities,problem.production_times,problem.setup_times,solution,problem.holding_costs,problem.setup_costs,1000)

In [22]:
run_tabu_search(problem,solution,sol_decode,sol_fit,10000,10000,4000)

Iteration: 2687, Best: 417979, Current: 449968:  27%|██▋       | 2687/10000 [00:09<00:26, 279.95it/s]

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/tomboerrigter/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/94/42yfz9lx6m97vnpccxd3mh640000gn/T/ipykernel_13764/1059788000.py", line 1, in <module>
    run_tabu_search(problem,solution,sol_decode,sol_fit,10000,10000,4000)
  File "/var/folders/94/42yfz9lx6m97vnpccxd3mh640000gn/T/ipykernel_13764/2807181239.py", line 33, in run_tabu_search
    current_solution, best_neighbour_fit, valid = run_tabu_iteration(problem,current_solution,neighbourhood_size,penalty,tabu_list)
                                                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/94/42yfz9lx6m97vnpccxd3mh640000gn/T/ipykernel_13764/2242974035.py", line -1, in run_tabu_iteration
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (mo

Iteration: 2687, Best: 417979, Current: 449968:  27%|██▋       | 2687/10000 [00:19<00:26, 279.95it/s]